---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)
df = pd.Series(doc)

In [123]:
mm = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [175]:
# 04/20/2009; 04/20/09; 4/20/09; 4/3/09; 4-13-82
d1 = df.str.extractall(r'(?P<mm>\d{1,2})(?:[-/])(?P<dd>\d{1,2})(?:[-/])(?P<yy>\d{4}|\d{2})(?:\D)')
# drop 2nd level index (match)
d1 = d1.reset_index(level=1, drop=True)

In [176]:
# Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
d2 = df.str.extractall(r"(?P<mm>"+'|'.join(mm) + r')(?:[a-z]*)(?:\s|-|. )(?P<dd>\d{0,2})(?:\s|\-|, )(?P<yy>\d{4})')
d2 = d2.reset_index(level=1, drop=True)

In [223]:
# 2 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
d3 = df.str.extractall(r'(?P<dd>\d{1,2})\s' + r"(?P<mm>"+'|'.join(mm) + r')(?:[a-z]*)(?:\s|-|. |, )(?P<yy>\d{4})')
d3 = d3.reset_index(level=1, drop=True)

In [272]:
# Mar 20th, 2009; Mar 21st, 2009; March 22nd, 2009
d4 = df.str.extractall(r'(?P<mm>'+'|'.join(mm) + r')(?:[a-z]*)\s(?P<dd>\d{0,2})(?:[a-z]{2}), (?P<yy>\d{4})')
d4 = d4.reset_index(level=1, drop=True)

In [226]:
# Feb 2009; Sep 2009; Oct 2010; December 1998; March, 2006; January 1993 (exclude the patterns from d3)
d5 = df[~df.index.isin(d3.index)].str.extractall(r"(?P<mm>"+'|'.join(mm) + r')(?:[a-z]*)(?:\s|, | )(?P<yy>\d{4})')
d5 = d5.reset_index(level=1, drop=True)

In [227]:
# 6/2008; 12/2009 (exclude the patterns from d1)
d6 = df[~df.index.isin(d1.index)].str.extractall(r'(?P<mm>\d{1,2})/(?P<yy>\d{4})')
d6 = d6.reset_index(level=1, drop=True)

In [228]:
# 1984 (exclude the patterns from d1 to d6)
d7 = df[~df.index.isin(d1.index | d2.index | d3.index | d4.index | d5.index | d6.index)].str.extractall(r'(?P<yy>\d{4})')
d7 = d7.reset_index(level=1, drop=True)

In [229]:
for i in range(1,8):
    for j in range(i+1, 8):
        print('d{} & d{}'.format(i, j), end='\t')
        print(set(eval('d{}'.format(i)).index) & set(eval('d{}'.format(j)).index))

d1 & d2	set()
d1 & d3	set()
d1 & d4	set()
d1 & d5	set()
d1 & d6	set()
d1 & d7	set()
d2 & d3	set()
d2 & d4	set()
d2 & d5	set()
d2 & d6	set()
d2 & d7	set()
d3 & d4	set()
d3 & d5	set()
d3 & d6	set()
d3 & d7	set()
d4 & d5	set()
d4 & d6	set()
d4 & d7	set()
d5 & d6	set()
d5 & d7	set()
d6 & d7	set()


In [276]:
len(d1) + len(d2) + len(d3) + len(d4) + len(d5) + len(d6) + len(d7)

500

In [277]:
set(range(500)) - (set(d1.index) | set(d2.index) | set(d3.index) | set(d4.index) | set(d5.index) | set(d6.index) | set(d7.index))

set()

In [232]:
from datetime import datetime

In [233]:
# 04/20/2009; 04/20/09; 4/20/09; 4/3/09; 4-13-82
def format_d1(strlist):
    # strlist in the format of [<yy>, <mm>, <dd>] 
    assert len(strlist) == 3 
    # yy without '19' padding
    if len(strlist[0]) == 2:
        strlist[0] = '19' + strlist[0]  
    # mm without '0' padding
    if len(strlist[1]) == 1:
        strlist[1] = '0' + strlist[1]
    # dd without '0' padding
    if len(strlist[2]) == 1:
        strlist[2] = '0' + strlist[2]
    return '{}-{}-{}'.format(strlist[0], strlist[1], strlist[2])

In [234]:
d1['dt'] = d1.apply(lambda row: datetime.strptime(format_d1([row['yy'], row['mm'], row['dd']]), '%Y-%m-%d'), axis=1)

In [246]:
# Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
# 2 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
def format_d2d3(strlist):
    # strlist in the format of [<yy>, <mm>, <dd>] 
    assert len(strlist) == 3 
    # yy without '19' padding
    if len(strlist[0]) == 2:
        strlist[0] = '19' + strlist[0]  
    # mm is %b format
    strlist[1] = strlist[1][:3]
    # dd without '0' padding
    if len(strlist[2]) == 1:
        strlist[2] = '0' + strlist[2]
    return '{}-{}-{}'.format(strlist[0], strlist[1], strlist[2])

In [242]:
d2['dt'] = d2.apply(lambda row: datetime.strptime(format_d2d3([row['yy'], row['mm'], row['dd']]), '%Y-%b-%d'), axis=1)

In [243]:
d3['dt'] = d3.apply(lambda row: datetime.strptime(format_d2d3([row['yy'], row['mm'], row['dd']]), '%Y-%b-%d'), axis=1)

In [275]:
# d4 is empty

In [278]:
# Feb 2009; Sep 2009; Oct 2010; December 1998; March, 2006; January 1993
def format_d5(strlist):
    # strlist in the format of [<yy>, <mm>] 
    assert len(strlist) == 2 
    # yy without '19' padding
    if len(strlist[0]) == 2:
        strlist[0] = '19' + strlist[0]  
    # mm is %b format
    strlist[1] = strlist[1][:3]
    return '{}-{}-{}'.format(strlist[0], strlist[1], '01')

In [280]:
d5['dt'] = d5.apply(lambda row: datetime.strptime(format_d5([row['yy'], row['mm']]), '%Y-%b-%d'), axis=1)

In [282]:
# 6/2008; 12/2009 (exclude the patterns from d1)
def format_d6(strlist):
    # strlist in the format of [<yy>, <mm>] 
    assert len(strlist) == 2 
    # yy without '19' padding
    if len(strlist[0]) == 2:
        strlist[0] = '19' + strlist[0]  
    # mm without '0' padding
    if len(strlist[1]) == 1:
        strlist[1] = '0' + strlist[1]
    return '{}-{}-{}'.format(strlist[0], strlist[1], '01')

In [283]:
d6['dt'] = d6.apply(lambda row: datetime.strptime(format_d6([row['yy'], row['mm']]), '%Y-%m-%d'), axis=1)

In [287]:
# 1984 (exclude the patterns from d1 to d6)
def format_d7(strlist):
    # strlist in the format of [<yy>] 
    assert len(strlist) == 1
    # yy without '19' padding
    if len(strlist[0]) == 2:
        strlist[0] = '19' + strlist[0]  
    return '{}-{}-{}'.format(strlist[0], '01', '01')

In [288]:
d7['dt'] = d7.apply(lambda row: datetime.strptime(format_d7([row['yy']]), '%Y-%m-%d'), axis=1)

In [301]:
dt = pd.Series(d1['dt']).append(d2['dt']).append(d3['dt']).append(d5['dt']).append(d6['dt']).append(d7['dt'])

In [313]:
dt_sorted = dt.sort_values()
answer = pd.Series(dt_sorted.index)

In [311]:
def date_sorter():  
    return pd.Series([9, 84, 2, 53, 28, 474, 153, 13, 129, 98, 111, 225, 31, 171, 191, 486, 415, 335, 36, 323, 405, 422, 375, 380, 345, 57, 481, 436, 104, 299, 162, 154, 402, 95, 73, 108, 156, 332, 182, 82, 351, 278, 214, 155, 223, 473, 49, 317, 11, 319, 40, 418, 165, 382, 370, 3, 50, 363, 219, 465, 237, 23, 342, 204, 258, 315, 27, 93, 17, 303, 488, 395, 283, 309, 419, 123, 19, 117, 232, 72, 189, 318, 493, 369, 239, 148, 105, 336, 6, 200, 81, 65, 434, 164, 378, 313, 495, 424, 398, 5, 254, 296, 75, 167, 21, 259, 499, 347, 150, 78, 340, 441, 267, 361, 221, 466, 134, 39, 197, 355, 430, 80, 444, 246, 85, 215, 263, 74, 403, 458, 16, 25, 127, 454, 70, 44, 59, 103, 112, 429, 88, 179, 358, 470, 205, 397, 294, 137, 295, 35, 438, 247, 209, 61, 107, 285, 175, 99, 455, 24, 421, 275, 48, 426, 489, 136, 30, 274, 10, 178, 1, 280, 447, 185, 228, 135, 69, 492, 199, 352, 8, 276, 230, 334, 96, 38, 368, 261, 404, 168, 29, 437, 423, 54, 485, 284, 68, 32, 349, 41, 63, 416, 55, 130, 116, 76, 462, 330, 37, 256, 390, 216, 174, 180, 476, 312, 265, 115, 71, 218, 202, 440, 385, 373, 210, 89, 149, 26, 7, 435, 482, 177, 157, 412, 22, 194, 14, 151, 233, 206, 245, 122, 94, 461, 226, 97, 91, 51, 33, 453, 67, 46, 322, 66, 399, 487, 138, 62, 211, 52, 269, 119, 100, 442, 310, 143, 301, 113, 298, 478, 272, 354, 0, 249, 192, 86, 172, 357, 331, 477, 300, 450, 163, 308, 196, 47, 133, 359, 64, 42, 409, 406, 238, 483, 193, 311, 140, 388, 56, 236, 372, 110, 248, 60, 181, 203, 326, 90, 169, 292, 479, 142, 4, 124, 324, 121, 131, 166, 468, 365, 213, 87, 353, 101, 333, 114, 459, 45, 338, 18, 222, 343, 20, 224, 12, 79, 387, 251, 120, 471, 77, 376, 432, 327, 384, 321, 212, 407, 266, 145, 201, 456, 260, 305, 420, 329, 392, 417, 190, 158, 443, 83, 374, 457, 125, 328, 159, 147, 195, 377, 367, 394, 494, 304, 446, 43, 262, 128, 102, 449, 184, 469, 452, 234, 362, 356, 144, 484, 291, 188, 414, 92, 350, 306, 241, 425, 281, 207, 126, 302, 146, 451, 498, 339, 250, 344, 346, 348, 496, 106, 118, 433, 270, 307, 173, 490, 410, 252, 314, 391, 277, 325, 264, 289, 160, 341, 132, 337, 428, 445, 497, 187, 183, 396, 271, 293, 400, 360, 491, 297, 371, 389, 386, 288, 379, 268, 472, 273, 287, 448, 176, 411, 408, 364, 242, 58, 467, 170, 15, 316, 240, 229, 217, 109, 227, 290, 460, 393, 282, 34, 220, 243, 208, 139, 320, 383, 480, 244, 286, 431, 279, 198, 463, 381, 366, 439, 255, 401, 475, 257, 152, 235, 464, 253, 427, 231, 141, 186, 161, 413])